In [ ]:
import json,datetime, plotly.express as px, re, pandas as pd, yaml, math
from os import listdir
from os.path import isfile, join, isdir

In [ ]:
dataset = []
with open("./scale_1.txt") as f:
    avg = []
    for line in f.readlines():
        line = line.split(",")
        dict = json.loads(",".join(line[1:]))
        avg.append(int(dict["result"]["no_sleep"]))
        
        dataset.append(json.loads(','.join(line[1:])))
    dataset = sorted(dataset, key=lambda x: x["config"]["timestep"])

check = set([1725660000 + i*300 for i in range(8640)])

for element in dataset:
    check.remove(element["config"]["timestep"])

timecheck = []
for i in check:
    timecheck.append((i-1725660000)/300)
print(sorted(timecheck))

asleep = [element["result"]["no_sleep"] for element in dataset]
time = [datetime.datetime.fromtimestamp(element["config"]["timestep"]) for element in dataset]
px.line(x=time, y=asleep).show()

In [ ]:
transceiver = pd.read_csv('transceiver.csv')
print(transceiver)

In [ ]:
path = '../devices/'
devices = {}
ports = ["SFP","SFP+","XFP","QSFP+","QSFP28","QSFP-DD"]
P_Port = {}
P_TRX_UP = {}

directories = [f for f in listdir(path) if isdir(join(path, f))]
for directory in directories:
    if not isfile(join(path, directory, 'power_model.yml')):
        continue
    else:
        with open(join(path, directory, 'power_model.yml')) as f:
            devices[directory] = yaml.load(f, Loader=yaml.FullLoader)

for port in ports:
    for device in devices:
        if port in devices[device]:
            for key in devices[device][port]:
                for speed in devices[device][port][key]:
                    if "P_PORT" in devices[device][port][key][speed].keys():
                        if math.isnan(devices[device][port][key][speed]["P_PORT"]):
                            continue
                        P_Port[port] = P_Port.get(port, []) + [devices[device][port][key][speed]["P_PORT"]]

                    if "P_TRX_UP" in devices[device][port][key][speed].keys():
                        if math.isnan(devices[device][port][key][speed]["P_TRX_UP"]):
                            continue
                        P_TRX_UP[port] = P_TRX_UP.get(port, []) + [devices[device][port][key][speed]["P_TRX_UP"]]
                        
for port in ports:
    if port not in P_Port.keys():
        P_Port[port] = 0
    else:
        P_Port[port] = sum(P_Port[port])/len(P_Port[port])
    if port not in P_TRX_UP.keys():
        P_TRX_UP[port] = 0
    else:
        P_TRX_UP[port] = sum(P_TRX_UP[port])/len(P_TRX_UP[port])

with open(f"./porttype.csv", "w") as f:
    f.write("form_factor,P_Port,P_TRX_UP\n")
    for port in P_Port:
        f.write(f"{port},{P_Port[port]},{P_TRX_UP[port]}\n")

In [ ]:
def parse_inventory(inventory_file):

    interface_data = {}

    with open(inventory_file, 'r') as file:
        for line in file:
            if "Transceiver" in line and  "Te" in line:
                name = line.split('\"')[1].split(" ")
                name = name[1][2:]
                print(name)
                desc = line.split(',')[1].split(':')[1].strip().strip('"')
                if len(desc.split(' ')) <= 1:
                    print(desc)
                    continue
                port_type = "unknown"
                trx_speed = desc.split(' ')[0]
                trx_phy = desc.split(' ')[1]
                # .. Parse the next line to extract the PID
                pid = next(file).split(',')[0].split(':')[1].strip()
                
            elif "subslot" in line and "transceiver" in line:
                name = line.split('\"')[1].split(" ")
                name = name[1] + "/" + name[3]
                desc = line.split(',')[1].split(':')[1].strip().strip('"')
                if len(desc.split(' ')) <= 1:
                    print(desc)
                    continue
                port_type = "unknown"
                trx_speed = desc.split(' ')[0]
                trx_phy = desc.split(' ')[1]
                # .. Parse the next line to extract the PID
                pid = next(file).split(',')[0].split(':')[1].strip()

            elif re.search('([0-9]*/){3}', line):
                # .. Parse the interface line
                # name = line.split(',')[0].split(':')[1].strip().strip('"').lower()
    
                name = re.search('([0-9]*/){3}[0-9]*', line).group()
                desc = line.split(',')[1].split(':')[1].strip().strip('"')

                if len(desc.split(' ')) <= 3:
                    print(line)
                    continue
                port_type = desc.split(' ')[1]
                trx_speed = desc.split(' ')[2]
                trx_phy = desc.split(' ')[3]
                # .. Parse the next line to extract the PID
                pid = next(file).split(',')[0].split(':')[1].strip()
            else:
                continue
            
            interface_data[name] = {
                    'description'   : desc,
                    'port_type'     : port_type,
                    'trx_speed'     : trx_speed,
                    'trx_phy'       : trx_phy,
                    'pid'           : pid
            }
    return interface_data

In [ ]:
path = './switch-device-inventory'
interface_data = {}
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
for file in onlyfiles:
    if "mgmt" in file:
        continue
    name = file.split(".")[0][3:]
    print(f"{path}/{file}")
    print(name)
    interface_data[name] = parse_inventory(f'{path}/{file}')
print(interface_data)

In [ ]:
power = []
port_power = []
trx_up_power = []

port = pd.read_csv("porttype.csv")
    

debug = False
missing = set()

for i, element in enumerate(dataset):
    if i%1000 == 0:
        print(datetime.datetime.fromtimestamp(element["config"]["timestep"]))
    power.append(0)
    port_power.append(0)
    trx_up_power.append(0)
    interfaces = []
    for edge in element["sleep_edges"]:
        if "bdi10" in edge[2][0]:
            continue
        split = edge[2][0].split("_")
        intf1 = split[0][-1] + "/" + "/".join(split[1:])
        if edge[0] not in interface_data.keys() or interface_data[edge[0]] == {}:
            if debug:
                print(f"no data {edge[0]} {intf1}")
            continue
        if intf1 not in interface_data[edge[0]].keys():
            if "0/" + intf1 not in interface_data[edge[0]].keys():
                if debug:
                    print(f"no data: {edge[0]} {intf1} {edge[2][0]}")
                continue
            interfaces.append((edge[0], interface_data[edge[0]]["0/" + intf1]["pid"]))
        else:
            interfaces.append((edge[0], interface_data[edge[0]][intf1]["pid"]))

    for edge in element["sleep_edges"]:
        if "bdi10" in edge[2][1]:
            continue
        split = edge[2][1].split("_")
        intf1 = split[0][-1] + "/" + "/".join(split[1:])
        if edge[1] not in interface_data.keys() or interface_data[edge[1]] == {}:
            if debug:
                print(f"no data {edge[1]} {intf1}")
            continue
        if intf1 not in interface_data[edge[1]].keys():
            if "0/" + intf1 not in interface_data[edge[1]].keys():
                if debug:
                    print(f"no data: {edge[1]} {intf1} {edge[2][1]}")
                continue
            interfaces.append((edge[1], interface_data[edge[1]]["0/" + intf1]["pid"]))
        else:
            interfaces.append((edge[1], interface_data[edge[1]][intf1]["pid"]))
    
    for intf in interfaces:
        if intf[1] not in transceiver["pid"].values:
            if debug:
                print(f"no data: {intf[1]}, {intf[0]}")
            missing.add(intf[1])
            continue
        power[i] += transceiver.loc[transceiver["pid"] == intf[1]]["power[W]"].values[0]
        form_factor = transceiver.loc[transceiver["pid"] == intf[1]]["form_factor"].values[0]
        port_power[i] += port[port["form_factor"] == form_factor]["P_Port"].values[0]
        trx_up_power[i] += port[port["form_factor"] == form_factor]["P_TRX_UP"].values[0]
    if debug:
        print(power[i])
        print(port_power[i])
        print(trx_up_power[i])
print(power)
print(port_power)
print(trx_up_power)
print(missing)

In [ ]:
print(sum(power)/len(power))
print(sum(port_power)/len(port_power))
print(sum(trx_up_power)/len(trx_up_power))

In [ ]:
with open("./transceiver_power.csv","w") as f:
    f.write("Time, Transceiver Power Saving\n")
    for i, element in enumerate(power):
        time = datetime.datetime.fromtimestamp(dataset[i]["config"]["timestep"])
        f.write(f"{time}, {power[i]:.2f}\n")

with open("./port_power.csv","w") as f:
    f.write("Time, Port Power Saving\n")
    for i, element in enumerate(port_power):
        time = datetime.datetime.fromtimestamp(dataset[i]["config"]["timestep"])
        f.write(f"{time}, {port_power[i]:.2f}\n")

with open("./trx_up_power.csv","w") as f:
    f.write("Time, TRX Up Power Saving\n")
    for i, element in enumerate(trx_up_power):
        time = datetime.datetime.fromtimestamp(dataset[i]["config"]["timestep"])
        f.write(f"{time}, {trx_up_power[i]:.2f}\n")

In [ ]:
def read_topo(topo_path):
    topo = {}
    with open(topo_path) as topo_file:
        for line in topo_file.readlines()[1:]:
            line = line.split(",")
            dev1 = line[0].replace("swi","")
            split = line[1].split("/")
            intf1 = split[0][-1] + "/" + "/".join(split[1:])
            if dev1 not in topo.keys():
                topo[dev1] = [intf1]
            else:
                topo[dev1].append(intf1)


            dev2 = line[2].replace("swi","")
            split = line[3].split("/")
            intf2 = split[0][-1] + "/" + "/".join(split[1:])
            if dev2 not in topo.keys():
                topo[dev2] = [intf2]
            else:
                topo[dev2].append(intf2)

    return topo

In [ ]:
path = './switch-device-inventory'
device_data = {}
device_data_port = {}
device_data_trx_up = {}
speed = {}
total = {}

missing_transceivers = set()
transceiver_cnt = {}
topo = read_topo("topology.csv")
port = pd.read_csv("porttype.csv")
count = 0
count_total = 0

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

for file in onlyfiles:
    if "mgmt" in file:
        continue
    name = file.split(".")[0][3:]
    device_data[name] = 0
    total[name] = 0
    device_data_port[name] = 0
    device_data_trx_up[name] = 0
    speed[name] = 0

    for element in interface_data[name]:
        pid = interface_data[name][element]["pid"]
        transceiver_cnt[pid] = transceiver_cnt.get(pid, 0) + 1
        count_total += 1
        if name in topo.keys() and element in topo[name]:
            count += 1
            if pid not in transceiver["pid"].values:
                missing_transceivers.add(pid)
                #print(f"no data: {pid}")
                continue
            
            device_data[name] += transceiver.loc[transceiver["pid"] == pid]["power[W]"].values[0]
            total[name] += transceiver.loc[transceiver["pid"] == pid]["power[W]"].values[0]
            form_factor = transceiver.loc[transceiver["pid"] == pid]["form_factor"].values[0]
            device_data_port[name] += port[port["form_factor"] == form_factor]["P_Port"].values[0]
            device_data_trx_up[name] += port[port["form_factor"] == form_factor]["P_TRX_UP"].values[0]
            speed[name] += int(transceiver.loc[transceiver["pid"] == pid]["speed"].values[0].replace("G",""))
        else:
            if pid not in transceiver["pid"].values:
                missing_transceivers.add(pid)
                #print(f"no data: {pid}")
                continue
            total[name] += transceiver.loc[transceiver["pid"] == pid]["power[W]"].values[0]
            speed[name] += int(transceiver.loc[transceiver["pid"] == pid]["speed"].values[0].replace("G",""))
            #print(f"not in topo: {name} {element}")


print(f"no_transceiver with data: {count},no_transceiver in topo: {sum([len(element) for element in topo.values()])}, total: {count_total}")
print(f"transceiver: {sum(device_data.values())}")
print(f"port: {sum(device_data_port.values())}")
print(f"trx_up: {sum(device_data_trx_up.values())}")
print(f"total: {sum(total.values())}")
print(f"speed: {sum(speed.values())}G")
print(sorted(transceiver_cnt.items(), key=lambda x: x[1], reverse=True))


In [ ]:
def parse_device(inventory_file):

    interface_data = {}
    pid = None
    with open(inventory_file, 'r') as file:
        for line in file:
            if "0/0/CPU0\"" in line or "\"Rack 0\"" in line or "\"Chassis\"" in line or "\"WS-C6504-E\"" in line or "\"CISCO7604\"" in line or "\"WS-C6503-E\"" in line or "\"1\"" in line:
                pid = next(file).split(',')[0].split(':')[1].strip()
                #print(pid)
    return pid

In [ ]:
path = './switch-device-inventory'
router_data = {}
router_cnt = {}
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
for file in onlyfiles:
    if "mgmt" in file:
        continue
    name = file.split(".")[0][3:]
    router_data[name] = parse_device(f'{path}/{file}')
print(router_data)
for name, dev in router_data.items():
    if dev == None:
        print(dev)
    else:
        router_cnt[dev] = router_cnt.get(dev, 0) + 1
print(sorted(router_cnt.items(), key=lambda x: x[1], reverse=True))

In [ ]:
with open("./device_trans_power.csv","w") as f:
    f.write("Device, Power\n")
    for element in device_data:
        f.write(f"{element}, {device_data[element]:.2f}\n")